# TODO

- Fix the timing out so I can add everything to the sqlite db.
- Finish grabbing the data for year 2017.
- Train IMDB sentiment analysis model from in the book below.
- Use that model and classify all the post titles, and comments.
- Use that with ../cryptocurrencies/ data and look for correlations between volume or returns


## references for later
- https://livebook.manning.com/#!/book/real-world-machine-learning/chapter-8/1
- http://praw.readthedocs.io/en/latest/tutorials/comments.html
- https://github.com/crazyguitar/pysheeet/blob/master/docs/notes/python-sqlalchemy.rst

In [2]:
import praw
from praw.models import MoreComments
from prawcore.exceptions import PrawcoreException

from reddit_stuff import reddit_login

from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

import pandas as pd



login = reddit_login()

reddit = praw.Reddit(**login)

print(reddit.user.me())

def make_tables():
    engine.execute('CREATE TABLE IF NOT EXISTS "Posts" ('
                   'id TEXT NOT NULL,'
                   'author TEXT, '
                   'domain TEXT,'
                   'created_utc INTEGER,'
                   'title TEXT,'
                   'view_count INTEGER,'
                   'ups INTEGER,'
                   'downs INTEGER,'
                   'score INTEGER,'
                   'edited BOOL,'
                   'num_crossposts INTEGER,'
                   'num_reports INTEGER,'
                   'banned_utc INTEGER,'
                   'removal_reason TEXT,'
                   'num_comments INTEGER,'
                   'pinned BOOL,'
                   'subreddit TEXT,'
                   'PRIMARY KEY (id));')


    engine.execute('CREATE TABLE IF NOT EXISTS "Comments" ('
                   'id TEXT NOT NULL,'
                   'author TEXT, '
                   'permalink TEXT,'
                   'created_utc INTEGER,'
                   'edited BOOL,'
                   'stickied BOOL,'
                   'ups INTEGER,'
                   'downs INTEGER,'
                   'score INTEGER,'
                   'removal_reason TEXT,'
                   'num_reports INTEGER,'
                   'parent TEXT,'
                   'subreddit TEXT,'
                   'body TEXT,'
                   'PRIMARY KEY (permalink));'
                  )

def insert_into_posts(values):
    engine.execute('INSERT OR IGNORE INTO "Posts" '
           '(\
           id, \
           author, \
           domain, \
           created_utc, \
           title, \
           view_count, \
           ups, \
           downs, \
           score, \
           edited, \
           num_crossposts, \
           num_reports, \
           banned_utc, \
           removal_reason, \
           num_comments, \
           pinned, \
           subreddit \
           ) '

           'VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', 
            (values))

def insert_into_comments(values):
    engine.execute('INSERT OR IGNORE INTO "Comments" '
           '(\
           id, \
           author, \
           permalink, \
           created_utc, \
           ups, \
           downs, \
           score, \
           edited, \
           removal_reason, \
           num_reports, \
           parent, \
           stickied, \
           subreddit, \
           body \
           ) '

           'VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)', 
            (values))
    
def get_post_values(submission):
    ID = submission.id
    author = str(submission.author)
    domain = str(submission.domain)
    created_utc = int(submission.created_utc or 0)
    title = submission.title
    view_count = int(submission.view_count or 0)
    ups = int(submission.ups or 0)
    downs = int(submission.downs or 0)
    score = int(submission.score or 0)
    edited = submission.edited
    num_crossposts = int(submission.num_crossposts or 0)
    num_reports = int(submission.num_reports or 0)
    banned_utc = int(submission.banned_at_utc or 0)
    removal_reason = str(submission.removal_reason)
    num_comments = int(submission.num_comments or 0)
    pinned = submission.pinned
    subreddit = str(submission.subreddit)

    print pd.to_datetime(created_utc, unit='s'), ID, subreddit

    values = (ID, author, domain, created_utc, title, view_count, 
              ups, downs, score, edited, num_crossposts,
              num_reports, banned_utc, removal_reason,
              
              num_comments, pinned, subreddit)

    return values



def get_comment_values(comment):
    author = str(comment.author)
    ID = str(comment.id)
    permalink = comment.permalink
    created_utc = int(comment.created_utc)
    edited = comment.edited
    ups = int(comment.ups)
    downs = int(comment.downs)
    score = int(comment.score)
    removal_reason = str(comment.removal_reason)
    num_reports = int(comment.num_reports or 0)
    parent = str(comment.parent())
    stickied = comment.stickied
    subreddit = str(comment.subreddit_name_prefixed)
    body = comment.body

    values = (author, ID, permalink, created_utc, edited,
              ups, downs, score, removal_reason, num_reports,
              parent, stickied, subreddit, body)

    return values

def get_subreddit_data(subreddit, date_start_utc, date_end_utc):
    for submission in subreddit.submissions(date_start_utc, date_end_utc): 
        values = get_post_values(submission)            
        insert_into_posts(values)

        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            values = get_comment_values(comment)
            insert_into_comments(values)
            
        

{'username': 'Picklebillharvey', 'client_secret': 'bxJOAK7UID5MtnKVhPxqoJTuD-U', 'password': 'mecp26', 'user_agent': 'crypto stuff by /u/Picklebillharvey', 'client_id': 'MZaMr8n8Ds6XMg'}
{'username': 'Picklebillharvey', 'client_secret': 'bxJOAK7UID5MtnKVhPxqoJTuD-U', 'password': 'mecp26', 'user_agent': 'crypto stuff by /u/Picklebillharvey', 'client_id': 'MZaMr8n8Ds6XMg'}
Picklebillharvey


In [ ]:
## !rm db.sqlite
engine = create_engine("sqlite:///db.sqlite")
conn = engine.connect()

conn.text_factory = lambda x: unicode(x, 'utf-8', 'ignore')

make_tables()

df = pd.DataFrame(pd.read_csv('crypto_subreddits.txt', header=None))

df.columns = ['subs','cointype']
df.iloc[:,0] = df.iloc[:,0].str.replace('/','')

bitcoin_subs = df.iloc[9:12,:].reset_index(drop=True).loc[:,'subs'].values

day_utc = 86400

start_date = 1483228800

date_start_utc = start_date
date_end_utc = start_date + day_utc

print bitcoin_subs

for sub in bitcoin_subs:
    print '----------------------------------------'
    print '----------------------------------------'
    print '----------------------------------------'
    print sub
    print '----------------------------------------'
    print '----------------------------------------'
    print '----------------------------------------'
    while date_end_utc <= 1514764800:
        
        print '----------------------------------------'
        print pd.to_datetime(date_start_utc, unit='s'), pd.to_datetime(date_end_utc, unit='s')
        print '----------------------------------------'
        
        subreddit = reddit.subreddit(sub)
        try:
            get_subreddit_data(subreddit, date_start_utc, date_end_utc)

        except PrawcoreException:
            print 'timeout...try again...'
            get_subreddit_data(subreddit, date_start_utc, date_end_utc)
        
        date_start_utc += day_utc
        date_end_utc += day_utc
    
    date_start_utc = start_date
    date_end_utc = start_date + day_utc



['ethtrader' 'ethereum' 'ethermining']
----------------------------------------
----------------------------------------
----------------------------------------
ethtrader
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
2017-01-01 00:00:00 2017-01-02 00:00:00
----------------------------------------
2017-01-01 23:38:00 5lhydb ethtrader
2017-01-01 22:28:32 5lhlpr ethtrader
2017-01-01 20:51:50 5lh3ao ethtrader
2017-01-01 19:31:26 5lgok3 ethtrader
2017-01-01 18:48:30 5lggjb ethtrader
2017-01-01 16:43:58 5lfute ethtrader
2017-01-01 15:15:38 5lfhrz ethtrader
2017-01-01 15:14:34 5lfhm9 ethtrader
2017-01-01 08:17:47 5leajf ethtrader
2017-01-01 07:06:40 5le2oz ethtrader
----------------------------------------
2017-01-02 00:00:00 2017-01-03 00:00:00
----------------------------------------
2017-01-02 23:01:13 5lo8va ethtrader
2017-01-02 22:19:53 5lo0vt ethtrader
2017-01-02 19:36: